***Вообще самая топовая площадка для стримов - одноклассники. Рил, тебя не грeбут за бан ворды, нет рекламы. Соперников в плане онлайна не будет, ты всегда будешь топ. Там нереально почти забаниться. Есть тоже смайлики, как на твиче. Подключи Донаты и все. Так же, тебе не будут кидать АП за музыку и т.д. Это идеальная площадка, я считаю.***


*какой-то умный чел*

## Импорты библиотек и файлов

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
main_path = ''

In [3]:
topics_df = pd.read_csv(main_path + 'data/topics.csv')
users_df = pd.read_csv(main_path + 'data/users.csv')
train_df = pd.read_csv(main_path + 'data/train.csv')
test_df = pd.read_csv(main_path + 'data/test.csv')

## Табличечки

### Ultra Users Data (uud)

Ultra Users Data (uud) хранит в себе сводные данные активностей пользователя:


*   likes - количество поставленных лайков
*   dislikes - количество поставленных дизлайков
*   reactions - количество поставленных реакций (train.csv)
*   test_reactions - количество поставленных реакций (test.csv)
*   total_reactions - количество реакций всего







In [4]:
user_likes = train_df[train_df['feedback'] == 'L'].groupby(by=['userId']).size().reset_index()
user_dislikes = train_df[train_df['feedback'] == 'D'].groupby(by=['userId']).size().reset_index()
user_test = test_df.groupby(by=['userId']).size().reset_index()

uud = users_df.merge(user_likes, on='userId', how='left')
uud = uud.merge(user_dislikes, on='userId', how='left')

uud = uud.drop(columns=['birthDate', 'gender', 'locationId'])
uud = uud.fillna(0).rename(columns={'0_x': 'likes',
                                    '0_y': 'dislikes'})

uud['reactions'] = uud['likes'] + uud['dislikes']

uud = uud.merge(user_test, on='userId', how='left').fillna(0)
uud = uud.rename(columns={0: 'test_reactions'})

uud['total_reactions'] = uud['reactions'] + uud['test_reactions']

In [5]:
uud.head()

,userId,likes,dislikes,reactions,test_reactions,total_reactions
0,2019458,1.0,1.0,2.0,0.0,2.0
1,96675,12.0,12.0,24.0,0.0,24.0
2,1378979,1.0,1.0,2.0,0.0,2.0
3,540187,2.0,3.0,5.0,3.0,8.0
4,1182831,2.0,2.0,4.0,0.0,4.0


### Ultra Topics Data (utd)

Ultra Topics Data (utd) содержит данные о постах, а точнее о реакциях на посты:


*  likes - количество поставленных лайков
*  dislikes - количество поставленных дизлайков
*  reactions - количество поставленных реакций (train.csv)



In [6]:
topic_likes = train_df[train_df['feedback'] == 'L'].groupby(by=['topicId']).size().reset_index()
topic_dislikes = train_df[train_df['feedback'] == 'D'].groupby(by=['topicId']).size().reset_index()

utd = topics_df['topicId'].to_frame().merge(topic_likes, on='topicId', how='left')
utd = utd.merge(topic_dislikes, on='topicId', how='left')

utd = utd.fillna(0).rename(columns={'0_x': 'likes',
                                    '0_y': 'dislikes'})

utd['reactions'] = utd['likes'] + utd['dislikes']

In [7]:
utd.head()

,topicId,likes,dislikes,reactions
0,190991,1.0,0.0,1.0
1,2718,287.0,443.0,730.0
2,49923,13.0,14.0,27.0
3,124548,4.0,0.0,4.0
4,45289,8.0,23.0,31.0


### User-Topic DataFrame (ut_df)

User-Topic DataFrame (ut_df) содержит информацию о постах, на которые отреагировал пользователь в **тестовой выборке**. Имеет вид: "Id пользователя" - [список постов]

In [8]:
ut_df = test_df.groupby(['userId'])['topicId'].apply(lambda x: x.values.tolist())

ut_df.head()

userId
0    [71366, 28, 126735, 78342, 44912, 9707, 6319, ...
1    [48810, 21078, 11916, 6169, 16519, 107, 60779,...
2    [20470, 8119, 20063, 4490, 3855, 66358, 1175, ...
3    [19424, 5523, 14970, 8003, 4750, 76414, 57860,...
4    [22525, 52245, 10401, 23133, 25490, 21679, 391...
Name: topicId, dtype: object

## А Н А Л И З

### А Н О М А Л И И

In [84]:
uud[(uud['test_reactions'] == 0) & (uud['reactions'] == 1)][['likes', 'dislikes']].value_counts()

likes  dislikes
1.0    0.0         116375
0.0    1.0          19232
dtype: int64

In [82]:
print(len(uud[(uud['test_reactions'] == 0) & (uud['likes'] == uud['dislikes'])]))
print(uud[(uud['test_reactions'] == 0) & (uud['likes'] == uud['dislikes'])][['likes', 'dislikes']].value_counts())

1535405
likes  dislikes
1.0    1.0         968874
2.0    2.0         300862
3.0    3.0         126790
4.0    4.0          61193
5.0    5.0          31991
6.0    6.0          18018
7.0    7.0          10524
8.0    8.0           6237
9.0    9.0           3869
10.0   10.0          2458
11.0   11.0          1506
12.0   12.0          1037
13.0   13.0           657
14.0   14.0           458
15.0   15.0           295
16.0   16.0           189
17.0   17.0           141
18.0   18.0            89
19.0   19.0            61
20.0   20.0            48
21.0   21.0            33
22.0   22.0            18
23.0   23.0            15
24.0   24.0            12
25.0   25.0            12
27.0   27.0             8
26.0   26.0             3
30.0   30.0             2
34.0   34.0             2
29.0   29.0             1
35.0   35.0             1
36.0   36.0             1
dtype: int64


Т.к. аномально много пользователей, которые поставили равное количество лайков и дизлайков, будем считать их за **ботов**

От этого будем отталкиваться при создании групп

### Создание бассейна

In [41]:
pool = uud[uud['test_reactions'] > 0]

### Разделение бассейна на группы

Первая группа: пользователи у которых 0 реакций в train.csv и одна реакция в test.csv (ставим как лайк)

In [42]:
group1 = pool[(pool['test_reactions'] == 1) & (pool['reactions'] == 0)]

pool = pool[~((pool['test_reactions'] == 1) & (pool['reactions'] == 0))]

Вторая группа: пользователи у которых большое соотношение лайков и дизлайков (больше чем 2:1)

Этим пользователям ставим те реакции, которых больше

In [43]:
group2 = pool[((abs(pool['likes'] - pool['dislikes']) / pool['reactions'] > 0.5)) & (pool['total_reactions'] >= 8)]

pool = pool[~(((abs(pool['likes'] - pool['dislikes']) / pool['reactions'] > 0.5)) & (pool['total_reactions'] >= 8))]

Третья группа: пользователи у которых известно, что все пропущенные реакции это либо только лайки, либо только дизлайки (боты, у которых легко выделить реакции)

In [44]:
group3 = pool[abs(pool['likes'] - pool['dislikes']) == pool['test_reactions']]

pool = pool[~(abs(pool['likes'] - pool['dislikes']) == pool['test_reactions'])]

Четвёртая группа: пользователи у которых сложнее выделить определённую реакцию (боты, у которых сложно выделить реакции)

In [45]:
group4 = pool[(pool['total_reactions'] % 2 == 0) & (abs(pool['dislikes'] - pool['likes']) < pool['test_reactions'])]
pool = pool[~((pool['total_reactions'] % 2 == 0) & (abs(pool['dislikes'] - pool['likes']) < pool['test_reactions']))]

Пятая группа: все остальные

In [46]:
group5 = pool

In [47]:
print(len(group1), group1['test_reactions'].sum())
print(len(group2), group2['test_reactions'].sum())
print(len(group3), group3['test_reactions'].sum())
print(len(group4), group4['test_reactions'].sum())
print(len(group5), group5['test_reactions'].sum())

15035 15035.0
14344 52438.0
689523 788999.0
203694 664765.0
29455 57565.0


### Получение ответов на каждую из групп

Первая группа: объяснено выше

In [64]:
group1['answers'] = 'L'
answers1 = group1[['userId', 'answers']]

Вторая группа: объяснено выше

In [65]:
answers2 = group2['likes'] > group2['dislikes']

answers2[answers2 == True] = 'L'
answers2[answers2 == False] = 'D'

answers2 = pd.concat([group2['userId'], answers2], axis=1).rename(columns={0: 'answers'})

Третья группа: где больше лайков, там ставим дизлайки и наоборот

In [66]:
answers3 = group3['likes'] < group3['dislikes']

answers3[answers3 == True] = 'L'
answers3[answers3 == False] = 'D'

answers3 = pd.concat([group3['userId'], answers3], axis=1).rename(columns={0: 'answers'})

Пятая группа: где больше лайков, там ставим лайки и наоборот

In [67]:
answers5 = group5['likes'] >= group5['dislikes']

answers5[answers5 == True] = 'L'
answers5[answers5 == False] = 'D'

answers5 = pd.concat([group5['userId'], answers5], axis=1).rename(columns={0: 'answers'})

In [68]:
answers_dict = {}

In [69]:
def cool_func(eval_df):
    reaction_dict = {}
    for index, row in tqdm(eval_df.iterrows(), total=eval_df.shape[0]):

        uid = int(row['userId'])
        reaction = str(row['answers'])
        posts = ut_df[uid]

        reaction_dict[uid] = dict(zip(posts, [reaction] * len(posts)))

    answers_dict.update(reaction_dict)
        


In [70]:
cool_func(answers1)
cool_func(answers2)
cool_func(answers3)

100%|██████████| 689523/689523 [00:43<00:00, 16011.30it/s]


Четвертая группа: ну тут алгоритм мне лень расписывать, basically на каком из постов больше дизлайков, на тот пост и поставят дизлайк (да, коряво объяснено, устно объяснить намного легче)

In [71]:
for index, row in tqdm(group4.iterrows(), total=group4.shape[0]):
    n_d = ((int(row['reactions']) + int(row['test_reactions'])) // 2) - int(row['dislikes'])

    uid = int(row['userId'])
    posts = ut_df[uid]

    pool = []
    for post in posts:
        reaction = int(utd[utd['topicId'] == post]['dislikes'])

        pool.append([post, reaction])
    pool = np.array(sorted(pool, key=lambda x: x[1], reverse=True))

    dislikes = pool[:n_d, 0]
    dislike_dict = dict(zip(dislikes, ['D'] * len(dislikes)))

    likes = pool[n_d:, 0]
    like_dict = dict(zip(likes, ['L'] * len(likes)))

    pred_reaction_dict = dict(like_dict)
    pred_reaction_dict.update(dislike_dict)
    answers_dict[uid] = pred_reaction_dict

100%|██████████| 203694/203694 [08:41<00:00, 390.39it/s]


In [72]:
cool_func(answers5)

100%|██████████| 29455/29455 [00:03<00:00, 9264.39it/s] 


In [73]:
print(len(answers_dict))
print(len(group1) + len(group2) + len(group3) + len(group4) + len(group5))
print(len(uud[uud['test_reactions'] > 0]))
print(test_df['userId'].nunique())

952051
952051
952051
952051


In [74]:
answers_list = [''] * len(test_df)

In [75]:
for i in tqdm(range(len(test_df))):
    uid = test_df.loc[i]['userId']
    tid = test_df.loc[i]['topicId']
    if uid in answers_dict:
        answers_list[i] = answers_dict[uid][tid]


100%|██████████| 1578802/1578802 [05:16<00:00, 4995.37it/s]


In [76]:
print(len(answers_list))
print(len(test_df))

1578802
1578802


In [77]:
print(answers_list[:100])

['D', 'L', 'L', 'L', 'D', 'D', 'D', 'L', 'D', 'D', 'L', 'D', 'L', 'D', 'D', 'L', 'L', 'D', 'L', 'D', 'L', 'L', 'L', 'L', 'D', 'L', 'L', 'D', 'L', 'D', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'L', 'L', 'L', 'D', 'L', 'D', 'D', 'L', 'L', 'D', 'D', 'L', 'D', 'L', 'L', 'D', 'D', 'D', 'D', 'L', 'D', 'D', 'L', 'D', 'D', 'L', 'L', 'L', 'D', 'L', 'D', 'D', 'L', 'L', 'D', 'L', 'D', 'L', 'D', 'L', 'D', 'D', 'L', 'L', 'L', 'D', 'L', 'L', 'L', 'L', 'D', 'L', 'D', 'D', 'L', 'D', 'D', 'L', 'L', 'L', 'D', 'D', 'L']


In [78]:
with open(main_path + 'final_answer.txt', 'w+') as f:
    f.write('\n'.join(answers_list))